In [1]:
from __future__ import print_function, division
import geopandas as gp
import pandas as pd
import pylab as pl
import os
import json


%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Load dataset
comp = pd.read_csv('/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9', \
    usecols=['Agency','Agency Name','Borough','Complaint Type','Community Board','Incident Zip','Location','Status','Unique Key', 'Resolution Description'])

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
comp.head()

,Unique Key,Agency,Agency Name,Complaint Type,Incident Zip,Status,Resolution Description,Community Board,Borough,Location
0,31911011,DOT,Department of Transportation,Street Condition,11224,Open,NaN,13 BROOKLYN,BROOKLYN,"(40.57343122248129, -73.99174247588253)"
1,31908754,CHALL,CHALL,Opinion for the Mayor,NaN,Email Sent,Your comments have been submitted to the Mayor...,0 Unspecified,Unspecified,NaN
2,31910423,DPR,Department of Parks and Recreation,Root/Sewer/Sidewalk Condition,11234,Open,The Department of Parks and Recreation usually...,18 BROOKLYN,BROOKLYN,"(40.62500363580505, -73.92072558378698)"
3,31909924,NYPD,New York City Police Department,Illegal Parking,11218,Assigned,Your complaint has been received by the Police...,12 BROOKLYN,BROOKLYN,"(40.634522428879706, -73.97479041437481)"
4,31913310,HRA,HRA Benefit Card Replacement,Benefit Card Replacement,NaN,Closed,The Human Resources Administration received yo...,0 Unspecified,Unspecified,NaN


In [47]:
origdata = comp.copy()

In [43]:
#All boroughs are represented plus unspecified
print('Unique Boroughs Count: ', len(comp.Borough.unique()))
print('Unique Boroughs list: ', comp.Borough.unique())

Unique Boroughs Count:  6
Unique Boroughs list:  ['BROOKLYN' 'Unspecified' 'STATEN ISLAND' 'MANHATTAN' 'BRONX' 'QUEENS']


In [48]:
origdata.rename(columns={'Community Board': 'Community_Board'}, inplace=True)
origdata.rename(columns={'Resolution Description': 'Resolution_Description'}, inplace=True)

In [49]:
#comp.head()

In [50]:
origdata.columns = [c.replace(' ', '_') for c in origdata.columns]
origdata.count()
  
# Check if all Boroughs and Community Districts are represented in the Data 
# there are 5 Boroughs & 59 Community Districts
print('Unique Boroughs Count: ', len(origdata.Borough.unique()))
print('Unique Boroughs list: ', origdata.Borough.unique())

print('Unique Community District Count: ', len(origdata.Community_Board.unique()))
print('Unique Community District List: ', origdata.Community_Board.unique())
# How many unique values do we have? 
#Some entries may be a multiple of the same entry, 77 Districts, should be 59

Unique Boroughs Count:  6
Unique Boroughs list:  ['BROOKLYN' 'Unspecified' 'STATEN ISLAND' 'MANHATTAN' 'BRONX' 'QUEENS']
Unique Community District Count:  77
Unique Community District List:  ['13 BROOKLYN' '0 Unspecified' '18 BROOKLYN' '12 BROOKLYN'
 '02 STATEN ISLAND' '08 MANHATTAN' '06 BROOKLYN' '10 BRONX' '06 MANHATTAN'
 '04 BROOKLYN' '04 QUEENS' '09 MANHATTAN' '14 BROOKLYN' '05 MANHATTAN'
 '02 MANHATTAN' '10 QUEENS' '07 QUEENS' '02 QUEENS' '11 BROOKLYN'
 '05 QUEENS' '05 BROOKLYN' '09 QUEENS' '12 MANHATTAN' '01 BROOKLYN'
 '11 MANHATTAN' '03 BRONX' '03 STATEN ISLAND' '08 BROOKLYN' '03 BROOKLYN'
 '07 MANHATTAN' '10 MANHATTAN' '04 BRONX' '01 MANHATTAN' '07 BRONX'
 '09 BRONX' '03 MANHATTAN' '13 QUEENS' '02 BRONX' '12 QUEENS' '16 BROOKLYN'
 '08 QUEENS' '04 MANHATTAN' '14 QUEENS' '10 BROOKLYN' '11 QUEENS'
 '06 BRONX' '08 BRONX' '05 BRONX' '15 BROOKLYN' '02 BROOKLYN' '12 BRONX'
 '03 QUEENS' '06 QUEENS' '01 STATEN ISLAND' '01 BRONX' '01 QUEENS'
 '17 BROOKLYN' '11 BRONX' 'Unspecified QUEENS'

In [51]:
#unique value in each
print (len(origdata['Community_Board'].unique()))
print (len(origdata['Borough'].unique()))

77
6


In [52]:
#Removing unspecified and invalid data
origdata = origdata[origdata['Community_Board'].str.contains('Unspecified') == False]
origdata = origdata[origdata['Borough'].str.contains('Unspecified') == False]

In [53]:
#Remove nan
origdata['Check'] = 'NaN'
#origdata = origdata.groupby(['Community_Board'], as_index = True).count()
origdata

,Unique_Key,Agency,Agency_Name,Complaint_Type,Incident_Zip,Status,Resolution_Description,Community_Board,Borough,Location,Check
0,31911011,DOT,Department of Transportation,Street Condition,11224,Open,NaN,13 BROOKLYN,BROOKLYN,"(40.57343122248129, -73.99174247588253)",NaN
2,31910423,DPR,Department of Parks and Recreation,Root/Sewer/Sidewalk Condition,11234,Open,The Department of Parks and Recreation usually...,18 BROOKLYN,BROOKLYN,"(40.62500363580505, -73.92072558378698)",NaN
3,31909924,NYPD,New York City Police Department,Illegal Parking,11218,Assigned,Your complaint has been received by the Police...,12 BROOKLYN,BROOKLYN,"(40.634522428879706, -73.97479041437481)",NaN
5,31914148,DOT,Department of Transportation,Highway Condition,10304,Open,The Department of Transportation requires 10 d...,02 STATEN ISLAND,STATEN ISLAND,"(40.60687536641399, -74.0854077221027)",NaN
7,31912611,NYPD,New York City Police Department,Noise - Commercial,10021,Open,Your complaint has been forwarded to the New Y...,08 MANHATTAN,MANHATTAN,"(40.769378795761845, -73.95793023233308)",NaN
8,31913909,NYPD,New York City Police Department,Noise - Commercial,11231,Closed,The Police Department responded to the complai...,06 BROOKLYN,BROOKLYN,"(40.682613541568664, -73.99323966551619)",NaN
9,31914856,NYPD,New York City Police Department,Noise - Commercial,11231,Closed,The Police Department responded to the complai...,06 BROOKLYN,BROOKLYN,"(40.682613541568664, -73.99323966551619)",NaN
10,31911308,NYPD,New York City Police Department,Blocked Driveway,10465,Open,Your complaint has been forwarded to the New Y...,10 BRONX,BRONX,"(40.820622268305286, -73.81001443391547)",NaN
11,31909392,TLC,Taxi and Limousine Commission,Taxi Complaint,10022,Open,The Taxi and Limousine Commission will contact...,06 MANHATTAN,MANHATTAN,"(40.75692417775419, -73.96997589542615)",NaN
12,31912284,NYPD,New York City Police Department,Blocked Driveway,11221,Closed,The Police Department issued a summons in resp...,04 BROOKLYN,BROOKLYN,"(40.69636767790248, -73.92873677664315)",NaN


In [17]:
#Clean up Columns
#origdata = origdata.drop(['Agency', 'Agency_Name', 'Incident_Zip'], axis = 1)

#origdata()

In [54]:
# Generate marker for unplausible Community Districts
# How do these districts look like? 
#def flag(data):
#    if re.search(r"Unspecified", data):
#        return 0
#    elif int(re.search(r"(\d\d*)\s\w+", data).group(1))>18:
#        return 0
#    else:
#       return 1

In [116]:
#Loop through rows. Split the Community_Board and convert to Integer: Matt Sloane
#threshold is 18 since 18 districts in each borough
#dfmi.loc[:,('one','second')] = value
#dfmi.loc.__setitem__((slice(None), ('one', 'second')), value)
for x in range(0, len(origdata)):
        if int(origdata['Community_Board'][x].split(' ')[0]) < 19:
            origdata['Check'][x] = 'Keep'
        else:
            origdata['Check'][x] = 'Delete'

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


KeyError: 1

In [ ]:
# Check for duplicates? Are these plausible?
origdata[origdata.duplicated(['Borough','Community_Board'])]

,Unique_Key,Agency,Agency_Name,Complaint_Type,Incident_Zip,Status,Resolution_Description,Community_Board,Borough,Location,Check,Complaint_Rank
9,31914856,NYPD,New York City Police Department,Noise - Commercial,11231,Closed,The Police Department responded to the complai...,06 BROOKLYN,BROOKLYN,"(40.682613541568664, -73.99323966551619)",NaN,3596602.0
17,31911881,NYPD,New York City Police Department,Illegal Parking,11230,Assigned,Your complaint has been received by the Police...,14 BROOKLYN,BROOKLYN,"(40.632158514338116, -73.97197392871156)",NaN,4456440.5
22,31910149,NYPD,New York City Police Department,Noise - Street/Sidewalk,10010,Assigned,Your complaint has been received by the Police...,05 MANHATTAN,MANHATTAN,"(40.7404174308271, -73.98482927836473)",NaN,3405952.5
26,31910446,DOT,Department of Transportation,Traffic Signal Condition,11385,Open,NaN,05 QUEENS,QUEENS,"(40.7004993066336, -73.90020149091094)",NaN,697955.0
28,31913287,DOT,Department of Transportation,Traffic Signal Condition,11385,Open,NaN,05 QUEENS,QUEENS,"(40.701968198847794, -73.88049664749286)",NaN,697955.0
29,31909985,NYPD,New York City Police Department,Illegal Parking,11415,Assigned,Your complaint has been received by the Police...,09 QUEENS,QUEENS,"(40.70491841899999, -73.82270083070199)",NaN,4456440.5
30,31908958,NYPD,New York City Police Department,Illegal Parking,11357,Closed,The Police Department responded to the complai...,07 QUEENS,QUEENS,"(40.779453281972245, -73.82335812210323)",NaN,4456440.5
31,31914364,DOT,Department of Transportation,Traffic Signal Condition,11415,Open,NaN,09 QUEENS,QUEENS,"(40.7093480453081, -73.82968656053104)",NaN,697955.0
33,31911348,NYPD,New York City Police Department,Blocked Driveway,11385,Closed,The Police Department responded to the complai...,05 QUEENS,QUEENS,"(40.69732784597591, -73.88980203373745)",NaN,7810165.5
35,31912735,NYPD,New York City Police Department,Illegal Parking,10022,Assigned,Your complaint has been received by the Police...,06 MANHATTAN,MANHATTAN,"(40.7618862594864, -73.9684214432797)",NaN,4456440.5


In [ ]:
origdata.drop_duplicates()

In [ ]:
# Rank Complaint Count
origdata['Complaint_Rank'] = origdata['Complaint_Type'].rank(ascending = False)
origdata

In [ ]:
origdata.dropna(axis = 0)

In [ ]:
#Section 4
#Load the Demographic data
demo = pd.read_csv('http://cosmo.nyu.edu/~fb55/PUI2016/data/Final_Demographics.csv')
demo.columns

In [ ]:
#http://stackoverflow.com/questions/28199524/best-way-to-count-the-number-of-rows-with-missing-values-in-a-pandas-dataframe
# Data Editing: Is it possible to replace missing values? Is it possible to use Complaint Type 
# to fill missings in Resolution Descriptor?
#this is for counting cells with missing values
sum(df.isnull().values.ravel())

In [ ]:
#data massaging
massdemo = demo[['cd_id', 'Total Population','Median household income (In 2014 Inflation Adjusted Dollars)']]
massdemo

In [ ]:
#Counting rows that have missing values somewhere:
#sum([True for idx,row in df.iterrows() if any(row.isnull())])

In [ ]:
#there are no repeated rows all 0
#df.isnull().sum()

In [ ]:
massdemo['Borough'] = 'NaN'
massdemo['CD_No'] = 'NaN'

for x in range(0, len(massdemo)):
        if massdemo['cd_id'][x][:2] == 'BX':
            massdemo['Borough'][x] = 'BRONX'
            massdemo['CD_No'][x] = massdemo['cd_id'][x][-2:]
        elif massdemo['cd_id'][x][:2] == 'BK':
            massdemo['Borough'][x] = 'BROOKLYN'
            massdemo['CD_No'][x] = massdemo['cd_id'][x][-2:]
        elif massdemo['cd_id'][x][:2] == 'MN':
            massdemo['Borough'][x] = 'MANHATTAN'
            massdemo['CD_No'][x] = massdemo['cd_id'][x][-2:]
        elif massdemo['cd_id'][x][:2] == 'QN':
            massdemo['Borough'][x] = 'QUEENS'
            massdemo['CD_No'][x] = massdemo['cd_id'][x][-2:]
        elif massdemo['cd_id'][x][:2] == 'SI':
            massdemo['Borough'][x] = 'STATEN ISLAND'
            massdemo['CD_No'][x] = massdemo['cd_id'][x][-2:]

In [ ]:
massdemo

In [ ]:
demo2 = massdemo.drop(['Borough', 'CD_No','cd_id'], axis = 1)
demo2

In [ ]:
#Load infra structure
df_infr = pd.read_csv('http://cosmo.nyu.edu/~fb55/PUI2016/data/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv')
df_infr.head()

In [ ]:
import re
df_infr['Borough'] = 'NaN'
df_infr['CD_Temp'] = 'NaN'
df_infr['CD_No'] = 'Nan'
df_infr['Community_Board'] = 'NaN'

for x in range(0,len(df_infr)):
    df_infr['CD_Temp'][x] = re.findall(r'\d+', df_infr['Qualifying Name'][x])  #find community district no. and assign
    df_infr['CD_No'][x] = df_infr['CD_Temp'][x][0] # convert list value to string value
    if df_infr['Qualifying Name'][x][8] == 'x':
        df_infr['Borough'][x] = 'BRONX'
    elif df_infr['Qualifying Name'][x][4] == 'M':
        df_infr['Borough'][x] = 'MANHATTAN'
    elif df_infr['Qualifying Name'][x][10] == 'y':
        df_infr['Borough'][x] = 'BROOKLYN'
    elif df_infr['Qualifying Name'][x][4] == 'Q':
        df_infr['Borough'][x] = 'QUEENS'
    elif df_infr['Qualifying Name'][x][4] == 'S':
        df_infr['Borough'][x] = 'STATEN ISLAND'

df_infr['Community Board'] = df_infr['CD_No'] + ' ' + df_infr['Borough']
df_infr

In [ ]:
# Assign High Connection and Low Connection Data Types (typology based on Kelsey Reid's / KDR276's approach)

connect_Low = df_infr.iloc[:,[6,9]]
connect_High = df_infr.iloc[:,[5,7,12]]

df_infr['Low Connection'] = connect_Low.sum(axis = 1)
df_infr['High Connection'] = connect_High.sum(axis = 1)

infr_Final = df_infr[['Community Board', 'Low Connection', 'High Connection']]
infr_Final.head(5)

In [ ]:
# Part 5 - Merge the DataFrames on the Community Board Field
merge_Temp = pd.merge(massdemo, demo2, on = ['Community Board'])
merge_Temp.head(5)

In [ ]:
final_Data = pd.merge(merge_Temp, infr_Final, on = ['Community Board'])
final_Data['Complaint Count'] = final_Data['Complaint Type']
final_Data = final_Data.drop(['Complaint Type'], axis = 1)
final_Data.head()

In [ ]:
fig = pd.scatter_matrix(final_Data, figsize = (15,15))